# Design Fashion of Clothes with Generative Adversarial Networks



![image](./docs/assets/clothes.png)
**Figure:** *dress image editing controlled via style images and segmentation masks with SEAN*

We propose semantic region-adaptive normalization (SEAN), a simple but effective building block for Generative Adversarial Networks conditioned on segmentation masks that describe the semantic regions in the desired output image. Using SEAN normalization, we can build a network architecture that can control the style of each semantic region individually, e.g., we can specify one style reference image per region. SEAN is better suited to encode, transfer, and synthesize style than the best previous method in terms of reconstruction quality, variability, and visual quality. We evaluate SEAN on datasets and report better quantitative metrics (e.g. FID, PSNR) than the current state of the art. SEAN also pushes the frontier of interactive image editing. We can interactively edit images by changing segmentation masks or the style for any given region. We can also interpolate styles from two reference images per region.



## Installation


This code requires PyTorch, python 3+ and Pyqt5. Please install dependencies by
```
pip install -r requirements.txt
```

This model requires a lot of memory and time to train. To speed up the training, we recommend using multi GPUs

## Dataset Preparation

This code uses [deepfashion] (https://mmlab.ie.cuhk.edu.hk/projects/DeepFashion.html) dataset. The prepared dataset can be directly downloaded [here] (https://mmlab.ie.cuhk.edu.hk/projects/DeepFashion/InShopRetrieval.html). After unzipping, put the entire deepfashion clothes(dfc) folder in the datasets folder. The complete directory should look like `./datasets/train/` and `./datasets/test/`.Download and unzip img_highres.zip from the deepfashion inshop dataset at $DATA_ROOT
We use DeepFashion dataset and provide our dataset split files, extracted keypoints files and extracted segmentation files for convience.

After the processing, you should have the dataset folder formatted like:


+ $DATA_ROOT
|   + datasets
|       + train (all training images)
|       |   - xxx.jpg
|       |     ...
|       +deepfashion\trainlabels (human parse of all training images)
|       |   - xxx.png
|       |     ...
|       + test (all test images)
|       |   - xxx.jpg
|       |     ...
|       + deepfashion\testlabels (human parse of all test images)
|       |   - xxx.png
|       |     ...
|       - fashion-resize-pairs-train.csv (paired poses for training)
|       - fashion-resize-pairs-test.csv (paired poses for test)
|       - fasion-resize-annotation-train.csv (keypoints for training images)
|       - fasion-resize-annotation-test.csv  (keypoints for test images)
|       - train.lst
|       - test.lst
|       - standard_test_anns.txt

1.Person images
Download person images from deep fasion dataset in-shop clothes retrival benchmark and download dataset split from Google Drive.Crop the images. Split the raw images into the train split (`./datasets/train/`) and the test split (`./datasets /test/`.).      

2.Keypoints files
Download train/test pairs and train/test key points annotations from Google Drive, including fashion-resize-pairs-train.csv,  fashion-resize-pairs-test.csv, fashion-resize-annotation-train.csv, fashion-resize-annotation-train.csv. Put these four files under the (./dataset/) directory.

3.Run python tools/generate_fashion_dataset.py --dataroot $DATAROOT to split the data.

4.Get human parsing. You can obtain the parsing by either:
Run off-the-shelf human parser  on`./ $DATA_ROOT/deepfashion/train/` and `./$DATA_ROOT/deepfashion/test. Name the output parses folder  as`./ $DATA_ROOT/deepfashion/train/trainlabels/` and`./ $DATA_ROOT/deepfashion/test/testlabels/` respectively`./ $DATA_ROOT.Download standard_test_anns.txt for fast visualization.


## Generating Images Using Pretrained Models

Once the dataset is prepared, the reconstruction results be got using pretrained models.


1.     Keypoints files

    Download train/test pairs and train/test key points annotations from Google Drive, including fashion-resize-pairs-train.csv,           fashion-resize-pairs-test.csv, fashion-resize-annotation-train.csv, fashion-resize-annotation-train.csv. Put these four files         under the deepfashion directory.
    

2. Generate the reconstruction results using the pretrained model.
	```bash
   python test.py --name deepfashion_pretrained --load_size 256 --crop_size 256 --dataset_mode custom --label_dir datasets/test/labels --image_dir datasets/test/images --label_nc 20 --no_instance --gpu_ids 0
    ```

3. The reconstruction images are saved at `./results/deepfashion_pretrained/` and the corresponding style codes are stored at `./styles_test/style_codes/`.

4. Pre-calculate the mean style codes for the UI mode. The mean style codes can be found at `./styles_test/mean_style_code/`.

	```bash
    python calculate_mean_style_code.py

## Training New Models

To train the new model, you need to specify the option `--dataset_mode custom`, along with `--label_dir [path_to_labels] --image_dir [path_to_images]`. You also need to specify options such as `--label_nc` for the number of label classes in the dataset, and `--no_instance` to denote the dataset doesn't have instance maps.


```bash
python train.py --name [experiment_name] --load_size 256 --crop_size 256 --dataset_mode custom --label_dir datasets/deepfashion/train/labels --image_dir datasets/deepfashion/train/images --label_nc 20 --no_instance --batchSize 32 --gpu_ids 0,1,2,3
```

If you only have single GPU with small memory, please use `--batchSize 2 --gpu_ids 0`.

## UI Introduction

We provide a convenient UI for the users to do some extension works. To run the UI mode, you need to:

1. run the step **Generating Images Using Pretrained Models** to save the style codes of the test images and the mean style codes. Or you can directly download the style codes from [here](https://drive.google.com/drive/folders/1GRz5HxNX4MKDkScb5s0JjkQHKjfc1Ztx). (Note: if you directly use the downloaded style codes, you have to use the pretrained model.

2. Put the visualization images of the labels used for generating in `./imgs/colormaps/` and the style images in `./imgs/style_imgs_test/`. Some example images are provided in these 2 folders. Note: the visualization image and the style image should be picked from `./datasets/deepfashion/test/vis/` and `./datasets/deepfashion/test/labels/`, because only the style codes of the test images are saved in `./styles_test/style_codes/`. If you want to use your own images, please prepare the images, labels and visualization of the labels in `./datasets/deepfashion/test/` with the same format, and calculate the corresponding style codes.

3. Run the UI mode

    ```bash
    python run_UI.py --name deepfashion_pretrained --load_size 256 --crop_size 256 --dataset_mode custom --label_dir datasets/deepfashion/test/labels --image_dir datasets/deepfashion/test/images --label_nc 20 --no_instance --gpu_ids 0
    ```
4. How to use the UI. Please check the detail usage of the UI from our .

	[![image](./docs/assets/UI.GIF)](https://youtu.be/0Vbj9xFgoUw)
